In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [18]:
import json
import csv
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import LabelEncoder
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split, GridSearchCV, StratifiedKFold, KFold
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestClassifier, RandomForestRegressor, GradientBoostingClassifier, GradientBoostingRegressor, ExtraTreesClassifier, ExtraTreesRegressor
from sklearn.svm import SVC, SVR
from sklearn.linear_model import LogisticRegression, LinearRegression, Ridge, Lasso, ElasticNet, SGDClassifier, SGDRegressor
from sklearn.neighbors import KNeighborsClassifier, KNeighborsRegressor
from sklearn.tree import DecisionTreeClassifier, DecisionTreeRegressor
from sklearn.neural_network import MLPClassifier, MLPRegressor
from xgboost import XGBClassifier, XGBRegressor
import re


In [2]:
!pip install pypandoc

In [26]:
import pypandoc

def convert_rtf_to_text(file_path):
    output = pypandoc.convert_file(json_file_path, 'plain', format='rtf')
    return output

# Example usage
json_file_path = '/kaggle/input/jason-parsing/algoparams_from_ui.json.rtf'
text = convert_rtf_to_text(json_file_path)

stripped_text = text.replace('\n', '')
text = stripped_text.replace(' ', '')
constraint_dict = json.loads(text)
print(type(constraint_dict))

<class 'dict'>


In [27]:

data = pd.read_csv('/kaggle/input/iris-dataset-dendrite/iris.csv')

# Split the dataset into features and target variable
X = data.drop(columns='species') 
y = data['species']
encoder = LabelEncoder()
y = encoder.fit_transform(y)

In [29]:


def feature_handling(X):
    # Removing Features
    if(constraint_dict['design_state_data']['feature_handling']['sepal_length']['is_selected'] == False):
        print("sepal_length False")
        X = X.drop('sepal_length', axis = 1)

    if(constraint_dict['design_state_data']['feature_handling']['sepal_width']['is_selected'] == False):
        print("False")
        X = X.drop('sepal_width', axis = 1)

    if(constraint_dict['design_state_data']['feature_handling']['petal_length']['is_selected'] == False):
        print("False")
        X = X.drop('petal_length', axis = 1)

    if(constraint_dict['design_state_data']['feature_handling']['petal_width']['is_selected'] == False):
        print("False")
        X = X.drop('petal_width', axis = 1)

    if(constraint_dict['design_state_data']['feature_handling']['sepal_length']['feature_details']['impute_with'] == 'Averageofvalues'):
        imputer = SimpleImputer(strategy="mean")
        X['sepal_length'] = imputer.fit_transform(X[['sepal_length']])
    elif(constraint_dict['design_state_data']['feature_handling']['sepal_length']['feature_details']['impute_with'] == 'custom'):
        imputer = SimpleImputer(strategy="constant", fill_value=int(constraint_dict['design_state_data']['feature_handling']['sepal_length']['feature_details']['impute_value']))
        X['sepal_length'] = imputer.fit_transform(X[['sepal_length']])

     # Imputing   
    if(constraint_dict['design_state_data']['feature_handling']['sepal_width']['feature_details']['impute_with'] == 'Averageofvalues'):
        imputer = SimpleImputer(strategy="mean")
        X['sepal_width'] = imputer.fit_transform(X[['sepal_width']])
    elif(constraint_dict['design_state_data']['feature_handling']['sepal_width']['feature_details']['impute_with'] == 'custom'):
        imputer = SimpleImputer(strategy="constant", fill_value=int(constraint_dict['design_state_data']['feature_handling']['sepal_width']['feature_details']['impute_value']))
        X['sepal_width'] = imputer.fit_transform(X[['sepal_width']])


    if(constraint_dict['design_state_data']['feature_handling']['petal_length']['feature_details']['impute_with'] == 'Averageofvalues'):
        imputer = SimpleImputer(strategy="mean")
        X['petal_length'] = imputer.fit_transform(X[['petal_length']])
    elif(constraint_dict['design_state_data']['feature_handling']['petal_length']['feature_details']['impute_with'] == 'custom'):
        imputer = SimpleImputer(strategy="constant", fill_value=int(constraint_dict['design_state_data']['feature_handling']['petal_length']['feature_details']['impute_value']))
        X['petal_length'] = imputer.fit_transform(X[['petal_length']])


    if(constraint_dict['design_state_data']['feature_handling']['petal_width']['feature_details']['impute_with'] == 'Averageofvalues'):
        imputer = SimpleImputer(strategy="mean")
        X['petal_width'] = imputer.fit_transform(X[['petal_width']])
    elif(constraint_dict['design_state_data']['feature_handling']['petal_width']['feature_details']['impute_with'] == 'custom'):
        imputer = SimpleImputer(strategy="constant", fill_value=int(constraint_dict['design_state_data']['feature_handling']['petal_width']['feature_details']['impute_value']))
        X['petal_width'] = imputer.fit_transform(X[['petal_width']])
    return X






In [30]:
X = feature_handling(X)

In [31]:

# Define a list of models with parameter grids for classification
classification_models = [
    ('RandomForestClassifier', RandomForestClassifier(), {
        'classifier__n_estimators': [50, 100],
        'classifier__max_depth': [None, 10, 20],
        'classifier__min_samples_split': [2, 5]
    }),
    ('GradientBoostingClassifier', GradientBoostingClassifier(), {
        'classifier__n_estimators': [50, 100],
        'classifier__learning_rate': [0.01, 0.1, 1],
        'classifier__max_depth': [3, 5]
    }),
    ('LogisticRegression', LogisticRegression(max_iter=1000), {
        'classifier__C': [0.1, 1, 10],
        'classifier__solver': ['liblinear', 'lbfgs']
    }),
    ('SVC', SVC(), {
        'classifier__kernel': ['linear', 'rbf'],
        'classifier__C': [0.1, 1, 10]
    }),
    ('SGDClassifier', SGDClassifier(max_iter=1000, tol=1e-3), {
        'classifier__loss': ['hinge', 'log'],
        'classifier__alpha': [0.0001, 0.001, 0.01]
    }),
    ('KNeighborsClassifier', KNeighborsClassifier(), {
        'classifier__n_neighbors': [3, 5, 7],
        'classifier__weights': ['uniform', 'distance']
    }),
    ('DecisionTreeClassifier', DecisionTreeClassifier(), {
        'classifier__max_depth': [None, 10, 20],
        'classifier__min_samples_split': [2, 5]
    }),
    ('ExtraTreesClassifier', ExtraTreesClassifier(), {
        'classifier__n_estimators': [50, 100],
        'classifier__max_depth': [None, 10, 20]
    }),
    ('XGBClassifier', XGBClassifier(), {
        'classifier__n_estimators': [50, 100],
        'classifier__learning_rate': [0.01, 0.1, 1],
        'classifier__max_depth': [3, 5]
    }),
    ('MLPClassifier', MLPClassifier(max_iter=1000), {
        'classifier__hidden_layer_sizes': [(50,), (100,)],
        'classifier__activation': ['tanh', 'relu'],
        'classifier__alpha': [0.0001, 0.001]
    })
]

# Define a list of models with parameter grids for regression
regression_models = [
    ('RandomForestRegressor', RandomForestRegressor(), {
        'regressor__n_estimators': [50, 100],
        'regressor__max_depth': [None, 10, 20],
        'regressor__min_samples_split': [2, 5]
    }),
    ('GradientBoostingRegressor', GradientBoostingRegressor(), {
        'regressor__n_estimators': [50, 100],
        'regressor__learning_rate': [0.01, 0.1, 1],
        'regressor__max_depth': [3, 5]
    }),
    ('LinearRegression', LinearRegression(), {}),
    ('Ridge', Ridge(), {
        'regressor__alpha': [0.1, 1, 10]
    }),
    ('Lasso', Lasso(), {
        'regressor__alpha': [0.1, 1, 10]
    }),
    ('ElasticNet', ElasticNet(), {
        'regressor__alpha': [0.1, 1, 10],
        'regressor__l1_ratio': [0.2, 0.5, 0.8]
    }),
    ('SVR', SVR(), {
        'regressor__kernel': ['linear', 'rbf'],
        'regressor__C': [0.1, 1, 10]
    }),
    ('SGDRegressor', SGDRegressor(max_iter=1000, tol=1e-3), {
        'regressor__loss': ['squared_loss', 'huber'],
        'regressor__alpha': [0.0001, 0.001, 0.01]
    }),
    ('KNeighborsRegressor', KNeighborsRegressor(), {
        'regressor__n_neighbors': [3, 5, 7],
        'regressor__weights': ['uniform', 'distance']
    }),
    ('DecisionTreeRegressor', DecisionTreeRegressor(), {
        'regressor__max_depth': [None, 10, 20],
        'regressor__min_samples_split': [2, 5]
    }),
    ('ExtraTreesRegressor', ExtraTreesRegressor(), {
        'regressor__n_estimators': [50, 100],
        'regressor__max_depth': [None, 10, 20]
    }),
    ('XGBRegressor', XGBRegressor(), {
        'regressor__n_estimators': [50, 100],
        'regressor__learning_rate': [0.01, 0.1, 1],
        'regressor__max_depth': [3, 5]
    }),
    ('MLPRegressor', MLPRegressor(max_iter=1000), {
        'regressor__hidden_layer_sizes': [(50,), (100,)],
        'regressor__activation': ['tanh', 'relu'],
        'regressor__alpha': [0.0001, 0.001]
    })
]


In [59]:
class_models_list = ['RandomForestClassifier', 'GBTClassifier', 'LogisticRegression', 'xg_boost', 'DecisionTreeClassifier', 'SVM', 'SGD', 'KNN', 'extra_random_trees', 'neural_network']
reg_models_list = ['RandomForestRegressor', 'GBTRegressor', 'LinearRegression', 'RidgeRegression', 'LassoRegression', 'ElasticNetRegression', 'xg_boost', 'DecisionTreeRegressor', 'SVM', 'SGD', 'KNN', 'extra_random_trees', 'neural_network']


In [33]:
""" Loading the dataset """ 
data = pd.read_csv('/kaggle/input/iris-dataset-dendrite/iris.csv')

# Split the dataset into features and target variable
X = data.drop(columns='species') 
y = data['species']
encoder = LabelEncoder()
y = encoder.fit_transform(y)
X = feature_handling(X)
shuffle = constraint_dict['design_state_data']['hyperparameters']['shuffle_grid']
random_state = constraint_dict['design_state_data']['hyperparameters']['random_state']
splits = constraint_dict['design_state_data']['hyperparameters']['num_of_folds']
njobs = constraint_dict['design_state_data']['hyperparameters']['parallelism']

# Cross-validation strategies
cv_classification = StratifiedKFold(n_splits=splits, shuffle=shuffle, random_state=random_state)
# Split the dataset into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Function to perform grid search for classification models
def perform_classification_grid_search(X_train, y_train, models):
    results = []
    for name, model, params in models:
        print(f"Running GridSearchCV for {name}")
        pipeline = Pipeline([
            ('scaler', StandardScaler()),
            ('classifier', model)
        ])
        grid_search = GridSearchCV(pipeline, param_grid=params, cv=cv_classification, scoring='accuracy', n_jobs=njobs, verbose=2)
        grid_search.fit(X_train, y_train)
        results.append((name, grid_search.best_params_, grid_search.best_score_))
    return results


# Perform grid search for classification models
classification_results = perform_classification_grid_search(X_train, y_train, classification_models)

# Print the results for classification
print("Classification Results:")
for result in classification_results:
    print(f"Model: {result[0]}")
    print(f"Best Parameters: {result[1]}")
    print(f"Best Cross-Validation Accuracy: {result[2]:.2f}")
    print("-" * 30)


    
    
# Function to perform grid search for regression models
def perform_regression_grid_search(X_train, y_train, models):
    results = []
    for name, model, params in models:
        print(f"Running GridSearchCV for {name}")
        pipeline = Pipeline([
            ('scaler', StandardScaler()),
            ('regressor', model)
        ])
        grid_search = GridSearchCV(pipeline, param_grid=params, cv=cv_regression, scoring='neg_mean_squared_error', n_jobs=njobs, verbose=2)
        grid_search.fit(X_train, y_train)
        results.append((name, grid_search.best_params_, grid_search.best_score_))
    return results

# Load regression data from CSV and split
data = pd.read_csv('/kaggle/input/iris-dataset-dendrite/iris.csv')    
X = data.drop(columns = 'species')
X = X.drop(columns= constraint_dict['design_state_data']['target']['target'])
y = data[constraint_dict['design_state_data']['target']['target']]
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
cv_regression = KFold(n_splits=splits, shuffle=shuffle, random_state=random_state)

# Perform grid search for regression models
regression_results = perform_regression_grid_search(X_train, y_train, regression_models)



# Print the results for regression
print("Regression Results:")
for result in regression_results:
    print(f"Model: {result[0]}")
    print(f"Best Parameters: {result[1]}")
    print(f"Best Cross-Validation MSE: {result[2]:.2f}")
    print("-" * 30)


Running GridSearchCV for RandomForestClassifier
Fitting 6 folds for each of 12 candidates, totalling 72 fits
Running GridSearchCV for GradientBoostingClassifier
Fitting 6 folds for each of 12 candidates, totalling 72 fits
Running GridSearchCV for LogisticRegression
Fitting 6 folds for each of 6 candidates, totalling 36 fits
Running GridSearchCV for SVC
Fitting 6 folds for each of 6 candidates, totalling 36 fits
Running GridSearchCV for SGDClassifier
Fitting 6 folds for each of 6 candidates, totalling 36 fits
Running GridSearchCV for KNeighborsClassifier
Fitting 6 folds for each of 6 candidates, totalling 36 fits


/opt/conda/lib/python3.10/site-packages/sklearn/linear_model/_stochastic_gradient.py:163: FutureWarning: The loss 'log' was deprecated in v1.1 and will be removed in version 1.3. Use `loss='log_loss'` which is equivalent.
  warnings.warn(
/opt/conda/lib/python3.10/site-packages/sklearn/linear_model/_stochastic_gradient.py:163: FutureWarning: The loss 'log' was deprecated in v1.1 and will be removed in version 1.3. Use `loss='log_loss'` which is equivalent.
  warnings.warn(
/opt/conda/lib/python3.10/site-packages/sklearn/linear_model/_stochastic_gradient.py:163: FutureWarning: The loss 'log' was deprecated in v1.1 and will be removed in version 1.3. Use `loss='log_loss'` which is equivalent.
  warnings.warn(
/opt/conda/lib/python3.10/site-packages/sklearn/linear_model/_stochastic_gradient.py:163: FutureWarning: The loss 'log' was deprecated in v1.1 and will be removed in version 1.3. Use `loss='log_loss'` which is equivalent.
  warnings.warn(
/opt/conda/lib/python3.10/site-packages/skle

Running GridSearchCV for DecisionTreeClassifier
Fitting 6 folds for each of 6 candidates, totalling 36 fits
Running GridSearchCV for ExtraTreesClassifier
Fitting 6 folds for each of 6 candidates, totalling 36 fits
Running GridSearchCV for XGBClassifier
Fitting 6 folds for each of 12 candidates, totalling 72 fits
[CV] END classifier__max_depth=None, classifier__min_samples_split=2, classifier__n_estimators=50; total time=   0.2s
[CV] END classifier__max_depth=None, classifier__min_samples_split=2, classifier__n_estimators=50; total time=   0.2s
[CV] END classifier__max_depth=None, classifier__min_samples_split=2, classifier__n_estimators=100; total time=   0.4s
[CV] END classifier__max_depth=None, classifier__min_samples_split=5, classifier__n_estimators=50; total time=   0.2s
[CV] END classifier__max_depth=None, classifier__min_samples_split=5, classifier__n_estimators=100; total time=   0.4s
[CV] END classifier__max_depth=10, classifier__min_samples_split=2, classifier__n_estimators=5

/opt/conda/lib/python3.10/site-packages/sklearn/model_selection/_validation.py:378: FitFailedWarning: 
18 fits failed out of a total of 36.
The score on these train-test partitions for these parameters will be set to nan.
If these failures are not expected, you can try to debug them by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
3 fits failed with the following error:
Traceback (most recent call last):
  File "/opt/conda/lib/python3.10/site-packages/sklearn/model_selection/_validation.py", line 686, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "/opt/conda/lib/python3.10/site-packages/sklearn/pipeline.py", line 405, in fit
    self._final_estimator.fit(Xt, y, **fit_params_last_step)
  File "/opt/conda/lib/python3.10/site-packages/sklearn/linear_model/_stochastic_gradient.py", line 1582, in fit
    self._validate_params()
  File "/opt/conda/lib/pytho

[CV] END classifier__learning_rate=1, classifier__max_depth=5, classifier__n_estimators=50; total time=   0.1s
[CV] END classifier__learning_rate=1, classifier__max_depth=5, classifier__n_estimators=50; total time=   0.1s
[CV] END classifier__activation=tanh, classifier__alpha=0.0001, classifier__hidden_layer_sizes=(50,); total time=   0.9s
[CV] END classifier__activation=tanh, classifier__alpha=0.0001, classifier__hidden_layer_sizes=(100,); total time=   0.8s
[CV] END classifier__activation=tanh, classifier__alpha=0.001, classifier__hidden_layer_sizes=(50,); total time=   0.7s
[CV] END classifier__activation=tanh, classifier__alpha=0.001, classifier__hidden_layer_sizes=(50,); total time=   0.8s
[CV] END classifier__activation=tanh, classifier__alpha=0.001, classifier__hidden_layer_sizes=(100,); total time=   0.8s
[CV] END classifier__activation=relu, classifier__alpha=0.0001, classifier__hidden_layer_sizes=(50,); total time=   0.5s
[CV] END classifier__activation=relu, classifier__alp

In [43]:
""" Loading the dataset """ 
data = pd.read_csv('/kaggle/input/iris-dataset-dendrite/iris.csv')

# Split the dataset into features and target variable
X = data.drop(columns='species') 
y = data['species']
encoder = LabelEncoder()
y = encoder.fit_transform(y)
X = feature_handling(X)
shuffle = constraint_dict['design_state_data']['hyperparameters']['shuffle_grid']
random_state = constraint_dict['design_state_data']['hyperparameters']['random_state']
splits = constraint_dict['design_state_data']['hyperparameters']['num_of_folds']
njobs = constraint_dict['design_state_data']['hyperparameters']['parallelism']

# Cross-validation strategies
cv_classification = StratifiedKFold(n_splits=splits, shuffle=shuffle, random_state=random_state)
# Split the dataset into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Function to perform grid search for classification models
def perform_classification_grid_search(X_train, y_train, models):
    print("*******Classification*********")
    results = []
    for name, model, params in models:
        model_name = re.sub(r"[^\w]", "", name)
        if(constraint_dict['design_state_data']['algorithms'][class_dict[model_name]]['is_selected'] == True):
#             print(model_name)
            print(f"Running GridSearchCV for {name}")
            pipeline = Pipeline([
                ('scaler', StandardScaler()),
                ('classifier', model)
            ])
            grid_search = GridSearchCV(pipeline, param_grid=params, cv=cv_classification, scoring='accuracy', n_jobs=njobs, verbose=2)
            grid_search.fit(X_train, y_train)
            results.append((name, grid_search.best_params_, grid_search.best_score_))
    return results


# Perform grid search for classification models
classification_results = perform_classification_grid_search(X_train, y_train, classification_models)

# Print the results for classification
print("Classification Results:")
for result in classification_results:
    print(f"Model: {result[0]}")
    print(f"Best Parameters: {result[1]}")
    print(f"Best Cross-Validation Accuracy: {result[2]:.2f}")
    print("-" * 30)


    
    
# Function to perform grid search for regression models
def perform_regression_grid_search(X_train, y_train, models):
    print("********Regression*********")
    results = []
    for name, model, params in models:
        
        model_name = re.sub(r"[^\w]", "", name)
        if(constraint_dict['design_state_data']['algorithms'][reg_dict[model_name]]['is_selected'] == True):
#             print(model_name)
            print(f"Running GridSearchCV for {name}")
            pipeline = Pipeline([
                ('scaler', StandardScaler()),
                ('regressor', model)
            ])
            grid_search = GridSearchCV(pipeline, param_grid=params, cv=cv_regression, scoring='neg_mean_squared_error', n_jobs=njobs, verbose=2)
            grid_search.fit(X_train, y_train)
            results.append((name, grid_search.best_params_, grid_search.best_score_))
    return results

# Load regression data from CSV and split
data = pd.read_csv('/kaggle/input/iris-dataset-dendrite/iris.csv')    
X = data.drop(columns = 'species')
X = X.drop(columns= constraint_dict['design_state_data']['target']['target'])
y = data[constraint_dict['design_state_data']['target']['target']]
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
cv_regression = KFold(n_splits=splits, shuffle=shuffle, random_state=random_state)

# Perform grid search for regression models
regression_results = perform_regression_grid_search(X_train, y_train, regression_models)



# Print the results for regression
print("Regression Results:")
for result in regression_results:
    print(f"Model: {result[0]}")
    print(f"Best Parameters: {result[1]}")
    print(f"Best Cross-Validation MSE: {result[2]:.2f}")
    print("-" * 30)


*******Classification*********
Classification Results:
********Regression*********
Running GridSearchCV for RandomForestRegressor
Fitting 6 folds for each of 12 candidates, totalling 72 fits
Regression Results:
Model: RandomForestRegressor
Best Parameters: {'regressor__max_depth': None, 'regressor__min_samples_split': 5, 'regressor__n_estimators': 100}
Best Cross-Validation MSE: -0.03
------------------------------


In [68]:
""" Loading the dataset """ 
data = pd.read_csv('/kaggle/input/iris-dataset-dendrite/iris.csv')

# Split the dataset into features and target variable
X = data.drop(columns='species') 
y = data['species']
encoder = LabelEncoder()
y = encoder.fit_transform(y)
X = feature_handling(X)


shuffle = constraint_dict['design_state_data']['hyperparameters']['shuffle_grid']
random_state = constraint_dict['design_state_data']['hyperparameters']['random_state']
splits = constraint_dict['design_state_data']['hyperparameters']['num_of_folds']
njobs = constraint_dict['design_state_data']['hyperparameters']['parallelism']




class_dict = {'RandomForestClassifier' : 'RandomForestClassifier',
              'GradientBoostingClassifier' : 'GBTClassifier', 
              'LogisticRegression' : 'LogisticRegression',
              'SVC' : 'SVM',
              'SGDClassifier' : 'SGD',
              'KNeighborsClassifier' : 'KNN',
              'DecisionTreeClassifier' : 'DecisionTreeClassifier',
              'ExtraTreesClassifier' : 'extra_random_trees', 
              'XGBClassifier' : 'xg_boost', 
              'MLPClassifier' : 'neural_network'}



# Cross-validation strategies
cv_classification = StratifiedKFold(n_splits=splits, shuffle=shuffle, random_state=random_state)
# Split the dataset into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Function to perform grid search for classification models
def perform_classification_grid_search(X_train, y_train, models):
    results = []
    for name, model, params in models:
        model_name = re.sub(r"[^\w]", "", name)
        print(f"Running GridSearchCV for {name}")
        pipeline = Pipeline([
            ('scaler', StandardScaler()),
            ('classifier', model)
        ])
        grid_search = GridSearchCV(pipeline, param_grid=params, cv=cv_classification, scoring='accuracy', n_jobs=njobs, verbose=2)
        grid_search.fit(X_train, y_train)
        results.append((name, grid_search.best_params_, grid_search.best_score_))
    return results





# # Perform grid search for classification models
classification_results = perform_classification_grid_search(X_train, y_train, classification_models)

# # Print the results for classification
# print("Classification Results:")
# for result in classification_results:
#     print(f"Model: {result[0]}")
#     print(f"Best Parameters: {result[1]}")
#     print(f"Best Cross-Validation Accuracy: {result[2]:.2f}")
#     print("-" * 30)



reg_dict = {'RandomForestRegressor':'RandomForestRegressor', 
            'GradientBoostingRegressor' : 'GBTRegressor', 
            'LinearRegression' : 'LinearRegression',
            'Ridge': 'RidgeRegression', 
            'Lasso' : 'LassoRegression', 
            'ElasticNet' : 'ElasticNetRegression', 
            'SVR' : 'SVM', 
            'SGDRegressor' : 'SGD', 
            'KNeighborsRegressor' : 'KNN', 
            'DecisionTreeRegressor' : 'DecisionTreeRegressor', 
            'ExtraTreesRegressor' : 'extra_random_trees',
            'XGBRegressor' : 'xg_boost', 
            'MLPRegressor' : 'neural_network'}
    
    
# # Function to perform grid search for regression models
# def perform_regression_grid_search(X_train, y_train, models):
#     results = []
#     for name, model, params in models:
#         print(f"Running GridSearchCV for {name}")
#         pipeline = Pipeline([
#             ('scaler', StandardScaler()),
#             ('regressor', model)
#         ])
#         grid_search = GridSearchCV(pipeline, param_grid=params, cv=cv_regression, scoring='neg_mean_squared_error', n_jobs=njobs, verbose=2)
#         grid_search.fit(X_train, y_train)
#         results.append((name, grid_search.best_params_, grid_search.best_score_))
#     return results

# # Load regression data from CSV and split
# data = pd.read_csv('/kaggle/input/iris-dataset-dendrite/iris.csv')    
# X = data.drop(columns = 'species')
# X = X.drop(columns= constraint_dict['design_state_data']['target']['target'])
# y = data[constraint_dict['design_state_data']['target']['target']]
# X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
# cv_regression = KFold(n_splits=splits, shuffle=shuffle, random_state=random_state)

# # Perform grid search for regression models
# regression_results = perform_regression_grid_search(X_train, y_train, regression_models)



# # Print the results for regression
# print("Regression Results:")
# for result in regression_results:
#     print(f"Model: {result[0]}")
#     print(f"Best Parameters: {result[1]}")
#     print(f"Best Cross-Validation MSE: {result[2]:.2f}")
#     print("-" * 30)


RandomForestClassifier
GradientBoostingClassifier
LogisticRegression
SVC
SGDClassifier
KNeighborsClassifier
DecisionTreeClassifier
ExtraTreesClassifier
XGBClassifier
MLPClassifier


In [9]:
total_algos = ['RandomForestClassifier', 'RandomForestRegressor', 'GBTClassifier', 'GBTRegressor', 
               'LinearRegression', 'LogisticRegression', 'RidgeRegression', 'LassoRegression', 
               'ElasticNetRegression', 'xg_boost', 'DecisionTreeRegressor', 'DecisionTreeClassifier', 
               'SVM', 'SGD', 'KNN', 'extra_random_trees', 'neural_network']

run_algos = ['RandomForestClassifier', 'GradientBoostingClassifier', 'LogisticRegression', 'SVC',
             'SGDClassifier', 'KNeighborsClassifier', 'DecisionTreeClassifier',
             'ExtraTreesClassifier', 'XGBClassifier', 'MLPClassifier']

In [37]:
class_dict = {'RandomForestClassifier' : 'RandomForestClassifier',
              'GradientBoostingClassifier' : 'GBTClassifier', 
              'LogisticRegression' : 'LogisticRegression',
              'SVC' : 'SVM',
              'SGDClassifier' : 'SGD',
              'KNeighborsClassifier' : 'KNN',
              'DecisionTreeClassifier' : 'DecisionTreeClassifier',
              'ExtraTreesClassifier' : 'extra_random_trees', 
              'XGBClassifier' : 'xg_boost', 
              'MLPClassifier' : 'neural_network'}

reg_dict = {'RandomForestRegressor':'RandomForestRegressor', 
            'GradientBoostingRegressor' : 'GBTRegressor', 
            'LinearRegression' : 'LinearRegression',
            'Ridge': 'RidgeRegression', 
            'Lasso' : 'LassoRegression', 
            'ElasticNet' : 'ElasticNetRegression', 
            'SVR' : 'SVM', 
            'SGDRegressor' : 'SGD', 
            'KNeighborsRegressor' : 'KNN', 
            'DecisionTreeRegressor' : 'DecisionTreeRegressor', 
            'ExtraTreesRegressor' : 'extra_random_trees',
            'XGBRegressor' : 'xg_boost', 
            'MLPRegressor' : 'neural_network'}

In [10]:
total_algos = ['RandomForestClassifier', 'RandomForestRegressor', 'GBTClassifier', 'GBTRegressor', 
               'LinearRegression', 'LogisticRegression', 'RidgeRegression', 'LassoRegression', 
               'ElasticNetRegression', 'xg_boost', 'DecisionTreeRegressor', 'DecisionTreeClassifier', 
               'SVM', 'SGD', 'KNN', 'extra_random_trees', 'neural_network']
reg_dict = {'RandomForestRegressor':'RandomForestRegressor', 
            'GradientBoostingRegressor' : 'GBTRegressor', 
            'LinearRegression' : 'LinearRegression',
            'Ridge': 'RidgeRegression', 
            'Lasso' : 'LassoRegression', 
            'ElasticNet' : 'ElasticNetRegression', 
            'SVR' : 'SVM', 
            'SGDRegressor' : 'SGD', 
            'KNeighborsRegressor' : 'KNN', 
            'DecisionTreeRegressor' : 'DecisionTreeRegressor', 
            'ExtraTreesRegressor' : 'extra_random_trees',
            'XGBRegressor' : 'xg_boost', 
            'MLPRegressor' : 'neural_network'}

In [17]:
for i in class_dict:
    print(i,"--", constraint_dict['design_state_data']['algorithms'][class_dict[i]]['is_selected'])

RandomForestClassifier -- False
GradientBoostingClassifier -- False
LogisticRegression -- False
SVC -- False
SGDClassifier -- False
KNeighborsClassifier -- False
DecisionTreeClassifier -- False
ExtraTreesClassifier -- False
XGBClassifier -- False
MLPClassifier -- False
